<a href="https://colab.research.google.com/github/tx1103mark/Fengshenbang-LM/blob/main/game_infer_script.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#!/bin/bash

cur_path=$(cd $(dirname $0); pwd)
echo "current path: $cur_path"
ALGO_PATH=$cur_path
export WANDB_DISABLED=true

base_path=/opt/huawei/explorer-env
base_path=/home/ma-user/work

model_path=$base_path/dataset/nlp_large_model_new/openLLM/glm-4-9b-chat/
# model_path=/opt/huawei/explorer-env/dataset/nlp_large_model_new/instruct_finetune/0614_searchllm_code_1ep
# dataset_dir=/home/ma-user/work/dataset/g00495223/instruct_tuning/data/sub_questions/
dataset_dir=/home/ma-user/work/dataset/nlp_large_data/z00421835/round1_training_data/
dataset_name=game
cutoff_len=4096


output_dir=/home/ma-user/work/dataset/nlp_large_model_new/z00421835/model_save/glm4_test \
script_path=$ALGO_PATH/src/train.py
ds_config_path=$ALGO_PATH/ds_config2.json


DISTRIBUTED_ARGS="--nproc_per_node  --nnodes $NNODES --node_rank $NODE_RANK --master_addr $MASTER_ADDR --master_port $MASTER_PORT"

echo "##DISTRIBUTED_ARGS:$DISTRIBUTED_ARGS"
    # --quantization_bit 4 \
export TOKENIZERS_PARALLELISM=false
# python -m torch.distributed.run --nproc_per_node=$NGPUS_PER_NODE --nnode=$NNODES --node_rank=$NODE_RANK --master_addr=$MASTER_ADDR --master_port=$MASTER_PORT
# python ${cur_path}/src/train_bash.py \
torchrun $script_path \
    --deepspeed ds_config2.json \
    --stage sft \
    --model_name_or_path $model_path \
    --do_train \
    --do_eval \
    --eval_steps 10 \
    --val_size 0.05 \
    --evaluation_strategy=steps \
    --dataset_dir  $dataset_dir\
    --dataset  $dataset_name \
    --template glm4 \
    --finetuning_type lora \
    --lora_target query_key_value \
    --output_dir $output_dir \
    --cache_dir /cache \
    --cutoff_len $cutoff_len \
    --flash_attn sdpa \
    --gradient_checkpointing true \
    --overwrite_output_dir \
    --per_device_train_batch_size 1 \
    --per_device_eval_batch_size 1 \
    --gradient_accumulation_steps 64 \
    --lr_scheduler_type cosine \
    --logging_steps 1 \
    --logging_dir log_dir \
    --save_strategy epoch \
    --learning_rate 5e-5 \
    --num_train_epochs 3.0 \
    --plot_loss \
    --fp16


In [ ]:
{
  "train_batch_size": "auto",
  "train_micro_batch_size_per_gpu": "auto",
  "gradient_accumulation_steps": "auto",
  "gradient_clipping": "auto",
  "zero_allow_untested_optimizer": true,
  "flops_profiler": {
        "enabled": false,
        "profile_step": 20,
        "module_depth": -1,
        "top_modules": 1,
        "detailed": true,
        "output_file": "profile.log"
   },
  "fp16": {
    "enabled": true,
    "loss_scale": 0,
    "loss_scale_window": 1000,
    "initial_scale_power": 16,
    "hysteresis": 2,
    "min_loss_scale": 1
  },
  "bf16": {
    "enabled": "auto"
  },
  "zero_optimization": {
    "stage": 2,
    "offload_optimizer": {
      "device": "cpu",
      "pin_memory": true
    },
    "allgather_partitions": true,
    "allgather_bucket_size": 5e8,
    "overlap_comm": true,
    "reduce_scatter": true,
    "reduce_bucket_size": 5e8,
    "contiguous_gradients": true,
    "round_robin_gradients": true
  }
}